In [22]:

!apt updateapt update
!pip install mdbtools-dev
!apt-get install mdbtools
!pip install pandas pyodbc
!pip install pymssql
!pip install tempfile2
!pip install japanize-matplotlib
import warnings
warnings.filterwarnings('ignore')

E: Invalid operation updateapt
ERROR: Could not find a version that satisfies the requirement mdbtools-dev (from versions: none)
ERROR: No matching distribution found for mdbtools-dev
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
mdbtools is already the newest version (1.0.0+dfsg-1).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.


In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
import sys
sys.path.append('/content/drive/MyDrive/Dev/py2/')
import numpy as np

import pandas as pd
from google.colab import files
import os
import subprocess
from io import StringIO  # StringIOを正しくインポート
import model.utility.k_analysis_util as k_a_util
import model.utility.k_analysis_util as k_analysis_util
from IPython.display import display
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib

In [25]:
def query_re_records_by_y(y):
  db_path = f"/content/drive/MyDrive/Dev/Kjvan/kjdb_prediction_cache_{y}.accdb"
  cmd = """select * from HorseResult
  """
 # where rr_r_id='{program_id}' and rr_r_race='{race_no}
  ret =  k_analysis_util.execute_mdb_sql(db_path, cmd)
  ret['rr_r_id'] = ret['rr_r_id'].astype(str)
  ret['rr_r_horse_id'] = ret['rr_r_horse_id'].astype(str)
  return ret


In [26]:
db_path = "/content/drive/MyDrive/Dev/Kjvan/kjdb_predict_result.accdb"
today = datetime.datetime.now()

In [27]:
sort_key ='pr_rank'
#sort_key ='pr_hE_pred'
ascending=True
cmd = f"select * from PredictResults"
df_pr = k_analysis_util.execute_mdb_sql(db_path, cmd)

Query output saved to: /tmp/tmpxh_8a8rq.txt


In [28]:
# まず'pr_program_id'を文字列型に変換
df_pr['pr_program_id'] = df_pr['pr_program_id'].astype(str)
df_re['rr_r_id'] = df_re['rr_r_id'].astype(str)
df_re['rr_r_horse_id'] = df_re['rr_r_horse_id'].astype(str)
df_re['rr_r_horse_name'] = df_re['rr_r_horse_name'].astype(str)
# その後、文字列抽出と数値変換を実行
df_pr = df_pr[df_pr['pr_program_id'].str[4:6].astype(int) >= today.month]

df_pr['pr_program_id'] = df_pr['pr_program_id'].astype(str)
df_pr['pr_horse_id'] = df_pr['pr_horse_id'].astype(str)
#df_pr=df_filtered

In [29]:
print(df_pr.head())

     pr_program_id pr_horse_id  pr_rank  pr_h1_pred  pr_h2_pred  pr_h3_pred  \
3243  2025020105    2022100485  0        0.0044      0.1457      0.2422       
3244  2025020105    2022100749  0        0.1445      0.5082      0.0320       
3245  2025020105    2022103030  0        0.0081      0.0393      0.0302       
3246  2025020105    2022103259  0        0.0135      0.1538      0.0993       
3247  2025020105    2022103340  0        0.0155      0.0228      0.1072       

      pr_h4_pred  pr_hE_pred  upd  pr_race pr_odds pr_vote  
3243  0.0         0.07505     1    1        1866    15      
3244  0.0         0.32635     1    1        0075    03      
3245  0.0         0.02370     1    1        1926    16      
3246  0.0         0.08365     1    1        0737    10      
3247  0.0         0.01915     1    1        1126    12      


In [30]:


# 表示オプションの設定
pd.set_option('display.max_colwidth', 1)  # 各列の最大表示幅を10に設定

# カスタムカラーマップの作成
colors = [
    (255/255, 255/255, 255/255),  # #FFF2F2 - ほぼ白
    # (255/255, 230/255, 230/255),  # #FFE6E6 - 非常に薄い赤 (90%)
    # (255/255, 217/255, 217/255),  # #FFD9D9 - 非常に薄い赤 (85%)
    (255/255, 204/255, 204/255),  # #FFCCCC - 非常に薄い赤 (80%)
    (255/255, 179/255, 179/255),  # #FFB3B3 - 薄い赤 (70%)
    (255/255, 153/255, 153/255),  # #FF9999 - 薄い赤 (60%)
    (255/255, 128/255, 128/255),  # #FF8080 - 薄い赤 (50%)
    (255/255, 102/255, 102/255),  # #FF6666 - 薄い赤 (40%)
    (255/255, 51/255, 51/255),    # #FF3333 - 薄い赤 (20%)
    (255/255, 0/255, 0/255),      # #FF0000 - 純粋な赤
    (204/255, 0/255, 0/255),      # #CC0000 - 赤
]
custom_cmap = sns.blend_palette(colors, as_cmap=True)
# bt_place、bt_distance、bt_track_cdでグループ化
grouped = df_pr.groupby(['pr_program_id', 'pr_race'])

In [68]:


# 各グループごとにヒートマップを描画
dict_re={}
for (program_id, race_no), group_df in grouped:
    try:
      y = program_id[:4]
      query = f"pr_rank.astype('int') != 0"
      group_df2 = group_df.query(query, engine='python')
      if(len(group_df2)):
        df_re = None
        if( y in  dict_re.keys()):
          df_re = dict_re[y]
        else:

          df_re = query_re_records_by_y(y)
          df_re['rr_r_id'] = df_re['rr_r_id'].astype(str)
          df_re['rr_r_horse_id'] = df_re['rr_r_horse_id'].astype(str)

          dict_re[y] = df_re


        # pr_rankでソート
        sorted_group_df = group_df2

        #sorted_group_df = group_df2.sort_values(sort_key, ascending = ascending)

        # 'pr_horse_id-pr_vote' の列を作成
        # print("--- マージ前のデータ型確認 ---")
        # print("\nsorted_group_df の型情報:")
        # print("pr_program_id の型:", sorted_group_df['pr_program_id'].dtype)
        # print("pr_horse_id の型:", sorted_group_df['pr_horse_id'].dtype)

        # print("\ndf_re の型情報:")
        # print("rr_r_id の型:", df_re['rr_r_id'].dtype)
        # print("rr_r_horse_id の型:", df_re['rr_r_horse_id'].dtype)
        sorted_group_df2 = pd.merge(sorted_group_df, df_re,
              left_on=['pr_program_id','pr_horse_id'],
              right_on=['rr_r_id','rr_r_horse_id'],
              how='left')
      #print(sorted_group_df2[['pr_program_id', 'pr_horse_id', 'rr_r_horse_name']])
      print(sorted_group_df2.head())
     # print(sorted_group_df2.columns)
      if(len(sorted_group_df2)):
          print(sorted_group_df2['rr_r_horse_name'])
          # sorted_group_df2['pr_horse_id-pr_vote'] = sorted_group_df2['rr_r_horse_name'].astype(str) + '-'+ sorted_group_df2['pr_vote'].astype(str).str.zfill(2) + '-'+ sorted_group_df2['pr_rank'].astype(str).str.zfill(2)
          # sorted_group_df2 = sorted_group_df2.sort_values(sort_key, ascending = ascending)
          # # ヒートマップのデータ作成
          # heatmap_data = sorted_group_df2[['pr_horse_id-pr_vote', 'pr_h1_pred', 'pr_h2_pred', 'pr_h3_pred', 'pr_h4_pred', 'pr_hE_pred']]
          # heatmap_data = heatmap_data.set_index('pr_horse_id-pr_vote')
          # heatmap_data = heatmap_data.rename(columns={
          #     'pr_h1_pred': 'mlp',
          #     'pr_h2_pred': 'cat',
          #     'pr_h3_pred': 'amt',
          #     'pr_h4_pred': 'bld',
          #     'pr_hE_pred': 'ens',
          # })
          # m = program_id[4:6]
          # plt.figure(figsize=(4, 4))  # 図のサイズ調整
          # ax = sns.heatmap(heatmap_data, annot=True, fmt=".1f", cmap='coolwarm', center=0, cbar=False)
          # ax.xaxis.set_ticks_position('top')     # 目盛りを上部に
          # ax.xaxis.set_label_position('top')     # ラベルを上部に
          # plt.title(f'{program_id}  Race: {race_no}')
          # plt.ylabel('')
          # plt.tight_layout()
          # plt.show()
    except Exception as e:
      print(e)


  pr_program_id pr_horse_id  pr_rank  pr_h1_pred  pr_h2_pred  pr_h3_pred  \
0  2025022310    2017110138  11       0.0291      0.6673      0.0          
1  2025022310    2018104708  1        0.0778      0.7077      0.0          
2  2025022310    2019100965  6        0.2781      0.7034      0.0          
3  2025022310    2019103586  12       0.0461      0.4941      0.0          
4  2025022310    2019105239  2        0.1315      0.8923      0.0          

   pr_h4_pred  pr_hE_pred  upd_x  pr_race  ... rr_m_blood12 rr_m_blood13  \
0  0.0         0.34820     1      11       ... NaN          NaN            
1  0.0         0.39275     1      11       ... NaN          NaN            
2  0.0         0.49075     1      11       ... NaN          NaN            
3  0.0         0.27010     1      11       ... NaN          NaN            
4  0.0         0.51190     1      11       ... NaN          NaN            

  rr_m_blood14  rr_a_deviation rr_a_deviation3f rr_a_race_count  rr_r_err  \
0 NaN    